## Tratamiento de datos

In [5]:
import pandas as pd
import numpy as np

### Parquet

In [6]:
df = pd.read_parquet('../datasets/vgsales.parquet')
df.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [7]:
df.set_index('Publisher', inplace=True)

df.head()

,Rank,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,,
Nintendo,1,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,2,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,3,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,4,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [8]:
df.drop('Rank', axis=1, inplace=True)
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [9]:
# check the nan values
df.isna().sum()

Name              0
Platform          0
Year            271
Genre             0
NA_Sales          0
EU_Sales          0
JP_Sales          0
Other_Sales       0
Global_Sales      0
dtype: int64

In [10]:
df.dropna(inplace=True)
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
Publisher,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16327 entries, Nintendo to Wanadoo
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Name          16327 non-null  object 
 1   Platform      16327 non-null  object 
 2   Year          16327 non-null  float64
 3   Genre         16327 non-null  object 
 4   NA_Sales      16327 non-null  float64
 5   EU_Sales      16327 non-null  float64
 6   JP_Sales      16327 non-null  float64
 7   Other_Sales   16327 non-null  float64
 8   Global_Sales  16327 non-null  float64
dtypes: float64(6), object(3)
memory usage: 1.2+ MB


### CSV

In [12]:
indie_games_csv = pd.read_csv('../datasets/indie-games-developers.csv')
indie_games_csv.head()

,Developer,City,Autonomous area,Country,Notable games,Notes
0,11 bit studios,Warsaw,NaN,Poland,AnomalyThis War of MineFrostpunk,NaN
1,ACE Team,Santiago,Santiago,Chile,Zeno ClashRock of Ages,NaN
2,Akupara Games,Los Angeles,California,United States,Whispering Willows The Metronomicon: Slay the ...,NaN
3,Alec Holowka,Winnipeg,Manitoba,Canada,Crayon Physics DeluxeI'm O.K – A Murder Simula...,NaN
4,Alientrap,Saskatoon,Saskatchewan,Canada,CapsizedApotheon,NaN


In [13]:
# comprobar si hay valores nulos 
indie_games_csv.isnull().sum()

Developer            0
City                15
Autonomous area     93
Country              0
Notable games        8
Notes              189
dtype: int64

In [14]:
indie_games_csv = indie_games_csv.drop(columns=['Notes', 'Notable games', 'Autonomous area'])
indie_games_csv.dropna(inplace=True)
indie_games_csv.head()

,Developer,City,Country
0,11 bit studios,Warsaw,Poland
1,ACE Team,Santiago,Chile
2,Akupara Games,Los Angeles,United States
3,Alec Holowka,Winnipeg,Canada
4,Alientrap,Saskatoon,Canada


In [15]:
video_games_csv = pd.read_csv('../datasets/video-games-developers.csv')
video_games_csv = video_games_csv.drop(columns=['Notes', 'Notable games, series or franchises', 'Administrative division', 'Est.'])
video_games_csv.head()

,Developer,City,Country
0,0verflow,Tokyo,Japan
1,11 bit studios,Warsaw,Poland
2,1C Company,Moscow,Russia
3,1-Up Studio,Tokyo,Japan
4,2K Czech,Brno,Czech Republic


### Unión de los csv

In [16]:
# unir los dos dataframes

video_games = pd.concat([indie_games_csv, video_games_csv], ignore_index=True)
# ordenar por nombre
video_games.sort_values(by='Developer', inplace=True)
video_games.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
0,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States


Filtramos por Developer para eliminar datos redundantes

In [17]:
video_games = video_games.drop_duplicates(subset='Developer', keep='first')
video_games.head(10)

,Developer,City,Country
185,0verflow,Tokyo,Japan
188,1-Up Studio,Tokyo,Japan
186,11 bit studios,Warsaw,Poland
187,1C Company,Moscow,Russia
189,2K Czech,Brno,Czech Republic
190,2K Games,Novato,United States
191,2K Sports,Novato,United States
192,343 Industries,Redmond,United States
193,38 Studios,Providence,United States
194,3D Realms,Garland,United States


Cuál es la ciudad que más vende?

---

## Unir CSV con Parquet

In [18]:
df = df.join(video_games.set_index('Developer'), on='Publisher')
df.head()

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,City,Country
Publisher,,,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74,Kyoto,Japan
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24,Kyoto,Japan
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82,Kyoto,Japan
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00,Kyoto,Japan
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37,Kyoto,Japan


In [19]:
df.dropna(inplace=True)
df.isnull().sum()

Name            0
Platform        0
Year            0
Genre           0
NA_Sales        0
EU_Sales        0
JP_Sales        0
Other_Sales     0
Global_Sales    0
City            0
Country         0
dtype: int64

### Paises con más ventas

In [20]:
df.groupby('Country')['Global_Sales'].sum().sort_values(ascending=False).head(10)

Country
United States     3790.87
Japan             2219.71
Europe             473.54
United Kingdom      66.98
France               7.86
Sweden               2.89
South Korea          2.64
Germany              2.13
Czech Republic       0.25
Norway               0.20
Name: Global_Sales, dtype: float64

### Publicadores con más ventas

In [21]:
df.groupby('Publisher')['Global_Sales'].sum().sort_values(ascending=False).head(10)

Publisher
Nintendo                       1784.43
Electronic Arts                1093.39
Activision                      721.41
Sony Computer Entertainment     607.28
Ubisoft                         473.54
THQ                             340.44
Sega                            270.70
Capcom                          199.95
Atari                           146.77
Square Enix                     144.73
Name: Global_Sales, dtype: float64

### Juego más vendido

In [22]:
df.loc[df['Global_Sales'].idxmax()]

,Name,Platform,Year,Genre,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,City,Country
Publisher,,,,,,,,,,,
Nintendo,Wii Sports,Wii,2006.0,Sports,41.49,29.02,3.77,8.46,82.74,Kyoto,Japan
Nintendo,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,40.24,Kyoto,Japan
Nintendo,Mario Kart Wii,Wii,2008.0,Racing,15.85,12.88,3.79,3.31,35.82,Kyoto,Japan
Nintendo,Wii Sports Resort,Wii,2009.0,Sports,15.75,11.01,3.28,2.96,33.00,Kyoto,Japan
Nintendo,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,31.37,Kyoto,Japan
...,...,...,...,...,...,...,...,...,...,...,...
Nintendo,Slide Adventure: Mag Kid,DS,2007.0,Action,0.00,0.00,0.01,0.00,0.01,Kyoto,Japan
Nintendo,Mario vs. Donkey Kong: Tipping Stars,WiiU,2015.0,Puzzle,0.00,0.00,0.01,0.00,0.01,Kyoto,Japan
Nintendo,Art Academy: Home Studio,WiiU,2015.0,Misc,0.00,0.00,0.01,0.00,0.01,Kyoto,Japan


### Plataforma con más ventas

In [23]:
df.groupby('Platform')['Global_Sales'].sum().sort_values(ascending=False).head(10)

Platform
PS2     833.39
Wii     779.08
PS3     726.98
DS      682.11
X360    624.82
PS      442.65
GBA     239.80
GB      235.14
PS4     215.72
3DS     209.66
Name: Global_Sales, dtype: float64

### Subir nuestro dataset a hugging face
Ya tenemos nuestro dataframe listo para subir a hugging face. 

Ahora sólo nos hace falta:
- Nuestro token de hugging (Disponible en access tokens)
- tener importado la librería `huggingface_hub` del cual usaremos la función `login`
- Importar la clase `Dataset`de la librería `datasets`

### Pasos a seguir:

1. Primero importamos las librerías que utilizaremos:
```python
from datasets import Dataset
from huggingface_hub import  login
```
2. Obtener nuestro token de acceso de hugging face:

Se puede obtener cuando hacemos click en  nuestro perfil -> access token -> create new token

!Importante conceder permisos de **escritura**¡

_OJO_
En este caso, hemos usado la función `getenv` de la librería `os` para tener nuestro token de acceso en una variable de entorno y que de esta forma, no sea accesible públicamente.

Sólo tenemos que llamar a la función `getenv` y pasarle el nombre que hemos especificado en nuestro archivo .env
```python

token = getenv('VARIABLE_CON_EL_TOKEN')
``` 

3. Parseamos nuestro dataframe a dataset 

Gracias a la clase `Dataset`, ya trae una función `from_pandas` la cual convierte nuestro dataframe en un dataset listo para publicarlo a nuestra cuenta de **huggingn face**

4. Loguearnos en hugging face

```python
login(token=token)
```

5. Publicar el dataset

```python
dataset.push_to_hub('nombre-del-repo')
```

In [ ]:
from datasets import Dataset
from huggingface_hub import  login
from os import getenv

#tu token de huggingface
token = getenv('TOKEN_HUGGINGFACE')

dataset = Dataset.from_pandas(df)

login(token=token)
dataset.push_to_hub('video-games-sales')


Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.31s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/ItzRoBeerT/video-games-sales/commit/0ccdd212cc2638b44d6043d1c7f7f32c2d6e347b', commit_message='Upload dataset', commit_description='', oid='0ccdd212cc2638b44d6043d1c7f7f32c2d6e347b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ItzRoBeerT/video-games-sales', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ItzRoBeerT/video-games-sales'), pr_revision=None, pr_num=None)

### Enlaces a kaggle
- [Videogames](https://www.kaggle.com/datasets/sagayaabinesh/videogames)
- [videogames-companies-regions](https://www.kaggle.com/datasets/andreshg/videogamescompaniesregions?select=indie-games-developers.csv)